In [2]:
import os
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
import os
import sys
import pandas as pd

In [3]:
import numpy as np 
import pandas as pd 
import os
import copy
import warnings
warnings.filterwarnings('ignore')
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
import matplotlib
import matplotlib.pylab as plt
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16,preprocess_input

In [4]:
W = 224
H = 224
label_to_class = {
    'MildDemented': 0,
    'ModerateDemented': 1,
    'NonDemented': 2,
    'VeryMildDemented':3
}
class_to_label = {v: k for k, v in label_to_class.items()}
n_classes = len(label_to_class)

In [5]:
def get_images(dir_name='C:\\Users\\User\\Desktop\\Dataset', label_to_class=label_to_class):
    Images = []
    Classes = []
    for j in ['/train','/test']:
        for label_name in os.listdir(dir_name+str(j)):
            cls = label_to_class[label_name]
            for img_name in os.listdir('/'.join([dir_name+str(j), label_name])):
                img = load_img('/'.join([dir_name+str(j), label_name, img_name]), target_size=(W, H))
                img = img_to_array(img)
                Images.append(img)
                Classes.append(cls)
    Images = np.array(Images, dtype=np.float32)
    Classes = np.array(Classes, dtype=np.float32)
    Images, Classes = shuffle(Images, Classes, random_state=0)
    return Images, Classes

In [6]:
Images, Classes = get_images()
Images.shape, Classes.shape

((6400, 224, 224, 3), (6400,))

In [7]:
indices_train, indices_test = train_test_split(list(range(Images.shape[0])), train_size=0.8, test_size=0.2, shuffle=False)
x_train = Images[indices_train]
y_train = Classes[indices_train]
x_test = Images[indices_test]
y_test = Classes[indices_test]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5120, 224, 224, 3), (5120,), (1280, 224, 224, 3), (1280,))

In [8]:
from keras.utils import np_utils
y_train = keras.utils.np_utils.to_categorical(y_train, n_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, n_classes)
y_train.shape, y_test.shape

((5120, 4), (1280, 4))

In [9]:
datagen_train = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rotation_range=30,     # rotation_range is a value in degrees (0-180), a range within which to randomly rotate pictures  
    width_shift_range=0.1, # width_shift and height_shift are ranges (as a fraction of total width or height) within which to randomly translate pictures vertically or horizontally          
    height_shift_range=0.1,# shifts images vertically(up or down).                 
    horizontal_flip=True,  # horizontal_flip is for randomly flipping half of the images horizontally                 
    vertical_flip=False,   # vertical is for randomly flipping half of the images vertically                 
    shear_range=0.2,       # shear_range is for randomly applying shearing transformations
    zoom_range=0.2         # zoom_range is for randomly zooming inside pictures
)
datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
)

In [10]:
img_rows, img_cols = 224,224
num_classes=4

x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
input_shape = (3, img_rows, img_cols)

train_dataset = x_train.astype('float32')
test_dataset = x_test.astype('float32')
train_dataset /= 255
test_dataset /= 255
print('train_dataset shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
train_labels = y_train
test_labels = y_test

train_dataset shape: (5120, 3, 224, 224)
5120 train samples
1280 test samples


In [11]:
from scipy.fftpack import fftn,fftshift
dims=train_dataset.shape
ft_train=[]
for i in range(dims[0]):
    ft_train.append(np.abs(fftshift(fftn(train_dataset[i,:,:]))))

dims=test_dataset.shape
ft_test=[]
for i in range(dims[0]):
    ft_test.append(np.abs(fftshift(fftn(test_dataset[i,:,:])))) #Don't forget to take abs!

ft_train=np.asarray(ft_train)
ft_test=np.asarray(ft_test)
print (ft_train.shape)
print (ft_test.shape)

(5120, 3, 224, 224)
(1280, 3, 224, 224)


In [12]:
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
import keras
model = keras.models.Sequential()
VGG = VGG16(include_top=False, input_shape=(W,H,3),pooling='avg')    
print(VGG.summary())
for layer in VGG.layers:
    layer.trainable=False
model.add(VGG)
model.add(Flatten())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
model.add(Dense(512,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(32,activation='relu',kernel_initializer='he_uniform'))

In [14]:
model.add(Dense(4,activation='softmax'))    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dense_3 (Dense)             (None, 4)                 132       
                                                                 
Total params: 15,047,268
Trainable params: 332,580
Non-t

In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [16]:
checkpoint_filepath = '{epoch:02d}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True)
earlystop = EarlyStopping(monitor = 'val_accuracy', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          mode='auto')
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy',
                              factor = 0.2,
                              patience = 3,
                              verbose = 1,
                              min_delta = 0.0001)
callbacks = [earlystop, model_checkpoint_callback, reduce_lr]
'''
   ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights 
   can be loaded later to continue the training from the state saved.
   Early stopping is a method that allows you to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold 
   out validation dataset.
   Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 
  'patience' number of epochs, the learning rate is reduced.
'''

"\n   ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights \n   can be loaded later to continue the training from the state saved.\n   Early stopping is a method that allows you to specify an arbitrary large number of training epochs and stop training once the model performance stops improving on a hold \n   out validation dataset.\n   Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a \n  'patience' number of epochs, the learning rate is reduced.\n"

In [17]:
import tensorflow as tf

In [18]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc')
]

In [20]:
dims=ft_train.shape
print (dims)
trainee=ft_train.reshape(5120,224,224,3)
dims=test_dataset.shape
testee=ft_test.reshape(1280,224,224,3)
print (trainee.shape)

(5120, 3, 224, 224)
(5120, 224, 224, 3)


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=METRICS)

epochs = 40

history=model.fit(trainee, train_labels, batch_size=32, verbose=1, epochs=40, validation_split = 0.1)

"""
history=model.fit(datagen_train.flow(ft_train,train_labels, batch_size=128,shuffle=True), 
                  epochs=epochs,validation_data=datagen_test.flow(ft_test,test_labels, 
                  batch_size=32,shuffle=True))
"""

Epoch 1/40
144/144 [==============================] - 872s 6s/step - loss: 6.2108 - accuracy: 0.7215 - auc: 0.6701 - val_loss: 1.1861 - val_accuracy: 0.7500 - val_auc: 0.6978
Epoch 2/40
144/144 [==============================] - 841s 6s/step - loss: 1.0467 - accuracy: 0.7625 - auc: 0.7846 - val_loss: 1.0220 - val_accuracy: 0.7627 - val_auc: 0.8159
Epoch 3/40
144/144 [==============================] - 863s 6s/step - loss: 1.0051 - accuracy: 0.7771 - auc: 0.8057 - val_loss: 0.9920 - val_accuracy: 0.7881 - val_auc: 0.8164
Epoch 4/40
144/144 [==============================] - 881s 6s/step - loss: 0.9751 - accuracy: 0.7858 - auc: 0.8143 - val_loss: 0.9828 - val_accuracy: 0.7847 - val_auc: 0.8160
Epoch 5/40
144/144 [==============================] - 936s 7s/step - loss: 0.9623 - accuracy: 0.7888 - auc: 0.8173 - val_loss: 1.0160 - val_accuracy: 0.7686 - val_auc: 0.8173
Epoch 6/40
144/144 [==============================] - 887s 6s/step - loss: 0.9553 - accuracy: 0.7916 - auc: 0.8198 - val_loss